<a href="https://colab.research.google.com/github/ALYAMBR/Notebooks/blob/master/CFT_test_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Загрузка файлов

Загрузим архив с данными с гуглдрайва и распакуем.

In [1]:
!cp /content/drive/MyDrive/Datasets/hw_cft.zip /content/hw_cft.zip

In [2]:
!unzip /content/hw_cft.zip

Archive:  /content/hw_cft.zip
  inflating: test.csv                
  inflating: train.csv               
  inflating: Variable_Definitions.csv  


In [4]:
import pandas as pd

Загрузим данные в память и посмотрим, что там есть.

In [5]:
var_defs = pd.read_csv("/content/Variable_Definitions.csv")

In [6]:
test_set = pd.read_csv("/content/test.csv")

In [7]:
train_set = pd.read_csv("/content/train.csv")

#Изучение данных

In [8]:
var_defs

,Column Name,Definition
0,TransactionId,Уникальный идентификатор транзакции на платформе
1,BatchId,Уникальное идентификатор присваевыемый пакету ...
2,AccountId,Уникальный идентификатор пользователя на платф...
3,SubscriptionId,Уникальный идентификатор идентифицирующий подп...
4,CustomerId,Уникальный идентификатор привязанный к аккаунту
5,CurrencyCode,Валюта страны
6,CountryCode,Цифровой географический код страны
7,ProviderId,Поставщик товара
8,ProductId,Идентификатор купленного товара
9,ProductCategory,Категория товара


Тут описание переменных. Фродовый статус транзакции это целевая переменная.

In [10]:
train_set

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,ClientProfile,TrasactionCategory
0,76871,36123,3957,887,4406,EUR,49,6,10,airtime,3,1000.0,1000,2018-11-15 02:18:49,2,0,4,3
1,73770,15642,4841,3829,4406,EUR,49,4,6,financial_services,2,-20.0,20,2018-11-15 02:19:08,2,0,0,2
2,26203,53941,4229,222,4683,EUR,49,6,1,airtime,3,500.0,500,2018-11-15 02:44:21,2,0,3,2
3,380,102363,648,2185,988,EUR,49,1,21,utility_bill,3,20000.0,21800,2018-11-15 03:32:55,2,0,3,1
4,28195,38780,4841,3829,988,EUR,49,4,6,financial_services,2,-644.0,644,2018-11-15 03:34:21,2,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79885,18019,129395,4841,3829,3795,EUR,49,4,6,financial_services,2,-5000.0,5000,2019-01-31 23:08:18,2,0,0,1
79886,102758,87434,2577,3326,2994,EUR,49,5,1,airtime,3,1000.0,1000,2019-01-31 23:44:44,4,0,3,1
79887,10757,114753,4841,3829,2994,EUR,49,4,6,financial_services,2,-50.0,50,2019-01-31 23:45:42,2,0,0,2
79888,104292,100831,2669,2002,3088,EUR,49,5,15,financial_services,3,45000.0,45000,2019-01-31 23:51:33,2,0,2,3


In [26]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79890 entries, 0 to 79889
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   TransactionId         79890 non-null  int64  
 1   BatchId               79890 non-null  int64  
 2   AccountId             79890 non-null  int64  
 3   SubscriptionId        79890 non-null  int64  
 4   CustomerId            79890 non-null  int64  
 5   CurrencyCode          79890 non-null  object 
 6   CountryCode           79890 non-null  int64  
 7   ProviderId            79890 non-null  int64  
 8   ProductId             79890 non-null  int64  
 9   ProductCategory       79890 non-null  object 
 10  ChannelId             79890 non-null  int64  
 11  Amount                79890 non-null  float64
 12  Value                 79890 non-null  int64  
 13  TransactionStartTime  79890 non-null  object 
 14  PricingStrategy       79890 non-null  int64  
 15  FraudResult        

Поизучаем данные. Затем напишем функцию, которая будет готовить данные к рабочему виду.

##FraudResult

Начнём с самого главного, посмотрим на соотношение классов.

In [36]:
train_set.FraudResult.value_counts()

0    79750
1      140
Name: FraudResult, dtype: int64

Тут всё как-то печально, на 80 тысяч записей всего 140 мошеннических.

3 атрибута имеют нечисловой тип. Выясним, что это за данные.

##CurrencyCode

In [28]:
train_set.CurrencyCode.unique()

array(['EUR'], dtype=object)

In [29]:
test_set.CurrencyCode.unique()

array(['EUR'], dtype=object)

Видим, что данный атрибут окажется при обучении абсолютно бесполезным, так как это константа. Поэтому просто удалим его.

##ProductCategory

In [30]:
train_set.ProductCategory.unique()

array(['airtime', 'financial_services', 'utility_bill', 'data_bundles',
       'tv', 'transport', 'ticket', 'movies', 'other'], dtype=object)

In [31]:
test_set.ProductCategory.unique()

array(['tv', 'financial_services', 'airtime', 'data_bundles',
       'utility_bill', 'ticket', 'transport', 'movies'], dtype=object)

In [35]:
train_set.ProductCategory.value_counts()

airtime               37715
financial_services    37590
utility_bill           1575
data_bundles           1563
tv                     1048
ticket                  206
movies                  168
transport                23
other                     2
Name: ProductCategory, dtype: int64

Тут всё хорошо. Во-первых, значения различные, во-вторых, в тренировочных данных есть все категории, которые есть в тестовом, что избавит от проблем неизвестных категорий.

##TransactionStartTime

In [51]:
trans_start_time = train_set.TransactionStartTime

In [52]:
trans_start_time

0        2018-11-15 02:18:49
1        2018-11-15 02:19:08
2        2018-11-15 02:44:21
3        2018-11-15 03:32:55
4        2018-11-15 03:34:21
                ...         
79885    2019-01-31 23:08:18
79886    2019-01-31 23:44:44
79887    2019-01-31 23:45:42
79888    2019-01-31 23:51:33
79889    2019-01-31 23:54:12
Name: TransactionStartTime, Length: 79890, dtype: object

Даты как минимум можно конвертировать в числа. Также можно сгенерировать фичи вроде секунд, времени суток. Возможно, это как-то поможет. Хотя без знания предметной области кажется, что время начала транзакции никак не влияет на её фрод статус.

## CountryCode

In [75]:
train_set.CountryCode.unique()

array([49])

In [76]:
test_set.CountryCode.unique()

array([49])

Просто удаляем, так как никакой информации этот признак не несёт.

##Amount

Можно сгенерировать фичу - дробная составляющая суммы. Так как есть небольшое количество транзакций, в которых сумма не круглое число. И это подозрительно, так как почти все транзакции имеют круглую сумму.

In [83]:
(train_set.Amount - train_set.Amount.astype(int)).value_counts()

 0.00    79618
-0.50      123
-0.25       15
-0.75       10
-0.20        8
         ...  
-0.40        1
-0.45        1
-0.90        1
-0.45        1
-0.90        1
Name: Amount, Length: 72, dtype: int64

##Функция препроцессинга

Напишем функцию, в которой к входному датасету будут применяться всякие преобразования.

In [ ]:
def prepare_data(data):
    data = data.drop(labels=["CurrencyCode", "CountryCode"])
    data.TransactionStartTime = data.TransactionStartTime.map(lambda x: pd.Timestamp(x)).astype(int)
    data.AmountTail = (data.Amount - data.Amount.astype(int))

## Разбиение обучающей выборки

Разобьём на X и Y для удобства.

In [12]:
train_X = train_set.drop(labels=["FraudResult"], axis=1)

In [14]:
train_Y = train_set.FraudResult

Разобьём на train, valid, test в отношении 3:1:1 .

In [18]:
SEED = 37

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
train_X, test_X, train_Y, test_Y = train_test_split(train_X, train_Y,
                                                    test_size=0.4,
                                                    random_state=SEED,
                                                    shuffle=True)

In [23]:
test_X, val_X, test_Y, val_Y = train_test_split(test_X, test_Y,
                                                test_size=0.5,
                                                random_state=SEED,
                                                shuffle=True)